In [1]:

import numpy as np
import cv2
import os
import sys
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as td
import torchvision.transforms as transforms
import torchvision.datasets as datasets



In [2]:

#Takes images in concat_data and flatten them into a 1D array
def flatten_images():
    images_dict = {'focused': [],
                    'happy': [],
                    'neutral': [],
                    'surprised': []}
    image_name_dict = {'focused': [],
                    'happy': [],
                    'neutral': [],
                    'surprised': []}
    
    for category in ['focused', 'happy', 'neutral', 'surprised']:
        #Get the list of images in the directory
        images = os.listdir("../concat_data/%s" % category)
        #Create an empty list to store the flattened images
        flattened_images = []
        #Iterate through the images
        
        for image in images:
            #check extension of the image
            if image.split(".")[-1] == "jpg":
                #Read the image
                img = cv2.imread("../concat_data/%s/%s" %(category, image), 0)
                #Flatten the image
                # img = img.flatten()
                #Add the flattened image to the list
                images_dict[category].append(img)
                image_name_dict[category].append(category + "_" + image)


    #Return the list of flattened images
    return images_dict, image_name_dict

In [3]:
# add images of each category into an array


images_dict, image_name_dict = flatten_images() 

# tokensize labels
labels = {'focused': 0,
          'happy': 1,
          'neutral': 2,
          'surprised': 3}


# concatencate all the data with respective labels
x = []
y = []
for key in images_dict:
    for image in images_dict[key]:
        x.append(image)
        y.append(labels[key])
x_name = []
y_name = []
for key in image_name_dict:
    for image_name in image_name_dict[key]:
        x_name.append(image_name)
        y_name.append(labels[key])
        
# split into training and valid/testing
x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size =0.30, random_state=42)
x_train_name, x_temp_name, y_train_name, y_temp_name = train_test_split(x_name, y_name, test_size =0.30, random_state=42)

# split x_temp and y_temp into validation and testing\
x_valid, x_test, y_valid, y_test = train_test_split(x_temp, y_temp, test_size =0.50, random_state=42)
x_valid_name, x_test_name, y_valid_name, y_test_name = train_test_split(x_temp_name, y_temp_name, test_size =0.50, random_state=42)




In [4]:
# Hyperparameters and settings
batch_size = 64
test_batch_size = 64
input_size = 1 # because there is only one channel 
output_size = 4
num_epochs = 10
learning_rate = 0.001



In [5]:
def find_image(arr_in):
    for directory in ['focused', 'happy', 'neutral', 'surprised']:
                images = os.listdir("../concat_data/%s" % directory)
                #Iterate through the images
                for image in images:
                    #check extension of the image
                    if image.split(".")[-1] == "jpg":
                        #Read the image
                        img = cv2.imread("../concat_data/%s/%s" %(directory, image), 0)
                        #Flatten the image
                        # img = img.flatten()
                        #Add the flattened image to the list
                        if np.array_equal(img, arr_in):
                            # 
                            print(image)
                            print(directory)
                            return image, directory
                            
         
    return None
def find_respective_images(tensor_in):
    # finds the image in the one of the directories
    # tensor_in: tensor
    # return: string
    dict_out = {'focused': [], 'happy': [], 'neutral': [], 'surprised': []}

    #Get the list of images in the directory
    for element in tensor_in:
        for element_name in element:
            numpy_tensor = element_name.numpy()
            image, directory = find_image(numpy_tensor)
            dict_out[directory].append(image)

            
    
    # print(tensor_in.numpy())

In [6]:

#Create data that can be fed into pytorch

#training
images_tensor = torch.tensor(x_train, dtype=torch.float32)
images_tensor = images_tensor.unsqueeze(1)
labels_tensor = torch.tensor(y_train, dtype=torch.long)

# validation
images_valid_tensor = torch.tensor(x_valid, dtype=torch.float32)
images_valid_tensor = images_valid_tensor.unsqueeze(1)
labels_valid_tensor = torch.tensor(y_valid, dtype=torch.long)

# testing
images_testing_tensor = torch.tensor(x_test, dtype=torch.float32)
images_testing_tensor = images_testing_tensor.unsqueeze(1)
labels_testing_tensor = torch.tensor(y_test, dtype=torch.long)

batch_size = 32


# Create a TensorDataset
# training
dataset_train = td.TensorDataset(images_tensor, labels_tensor)
data_loader = td.DataLoader(dataset_train, batch_size=batch_size, shuffle=True)


# validation
dataset_valid = td.TensorDataset(images_valid_tensor, labels_valid_tensor)
valid_loader = td.DataLoader(dataset_valid, batch_size=batch_size, shuffle=True)


# testing
dataset_test = td.TensorDataset(images_testing_tensor, labels_testing_tensor)
test_loader = td.DataLoader(dataset_test, batch_size=batch_size, shuffle=True)


#

C:\Users\Luis\AppData\Local\Temp\ipykernel_17924\2895347494.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:278.)
  images_tensor = torch.tensor(x_train, dtype=torch.float32)


In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(12 * 12 * 64, 1000),
            nn.ReLU(inplace=True),
            nn.Linear(1000, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        # conv layers
        x = self.conv_layer(x)
        # print(x.shape)
        # flatten
        x = x.view(x.size(0), -1)
        # print(x.shape)
        # fc layer
        x = self.fc_layer(x)
        return x


In [8]:
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
total_step = len(data_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(data_loader):
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())
        # Backprop and optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Train accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)
        # print(i)
        if (i + 1) % 10 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
            .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
            (correct / total) * 100))

Epoch [1/10], Step [10/44], Loss: 1.0750, Accuracy: 53.12%
Epoch [1/10], Step [20/44], Loss: 0.7600, Accuracy: 71.88%
Epoch [1/10], Step [30/44], Loss: 0.7878, Accuracy: 68.75%
Epoch [1/10], Step [40/44], Loss: 0.6883, Accuracy: 65.62%
Epoch [2/10], Step [10/44], Loss: 0.4698, Accuracy: 81.25%
Epoch [2/10], Step [20/44], Loss: 0.2608, Accuracy: 90.62%
Epoch [2/10], Step [30/44], Loss: 0.4317, Accuracy: 87.50%
Epoch [2/10], Step [40/44], Loss: 0.4328, Accuracy: 81.25%
Epoch [3/10], Step [10/44], Loss: 0.3652, Accuracy: 84.38%
Epoch [3/10], Step [20/44], Loss: 0.1320, Accuracy: 93.75%
Epoch [3/10], Step [30/44], Loss: 0.3434, Accuracy: 87.50%
Epoch [3/10], Step [40/44], Loss: 0.3348, Accuracy: 87.50%
Epoch [4/10], Step [10/44], Loss: 0.1034, Accuracy: 96.88%
Epoch [4/10], Step [20/44], Loss: 0.0861, Accuracy: 93.75%
Epoch [4/10], Step [30/44], Loss: 0.2213, Accuracy: 90.62%
Epoch [4/10], Step [40/44], Loss: 0.3388, Accuracy: 90.62%
Epoch [5/10], Step [10/44], Loss: 0.1395, Accuracy: 93.7

In [10]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in valid_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        # print(predicted)
        # print(labels)
        # print("///////")
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Test Accuracy of the model on the 10000 test images: {} %'
    .format((correct / total) * 100))

Test Accuracy of the model on the 10000 test images: 90.0 %


In [11]:
print("test:", x_test_name)
print("validation:", x_valid_name)
print("training:", x_train_name)
print(len(x_train_name))
print(len(x_valid_name))
print(len(x_test_name))

test: ['happy_246_chicago-faces.jpg', 'surprised_370_face-expression-recognition-dataset-jonathan-oheix.jpg', 'neutral_196_chicago-faces.jpg', 'happy_25_chicago-faces.jpg', 'happy_258_chicago-faces.jpg', 'neutral_371_chicago-faces.jpg', 'focused_121_angry-face-expression-recognition-dataset-jonathan-oheix.jpg', 'happy_69_kaggle__zawarkhan69__human-facial-expression-dataset.jpg', 'focused_50_UTK-sanjaya.jpg', 'happy_90_chicago-faces.jpg', 'surprised_326_face-expression-recognition-dataset-jonathan-oheix.jpg', 'focused_47_angry-face-expression-recognition-dataset-jonathan-oheix.jpg', 'happy_11_chicago-faces.jpg', 'focused_34_MMA-FACIAL-EXPRESSION-mahmoud.jpg', 'focused_22_UTK-sanjaya.jpg', 'surprised_367_face-expression-recognition-dataset-jonathan-oheix.jpg', 'happy_158_chicago-faces.jpg', 'focused_29_MMA-FACIAL-EXPRESSION-mahmoud.jpg', 'happy_276_chicago-faces.jpg', 'surprised_37_face-expression-recognition-dataset-jonathan-oheix.jpg', 'neutral_3_chicago-faces.jpg', 'surprised_242_face

In [12]:
for name in x_test_name:
    if (name in x_valid_name) or (name in x_train_name):
        print(name)
        print("error")
for name in x_valid_name:
    if (name in x_test_name) or (name in x_train_name):
        print(name)
        print("error")
for name in x_train_name:
    if (name in x_valid_name) or (name in x_test_name):
        print(name)
        print("error")

In [13]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        # print(predicted)
        # print(labels)
        # print("///////")
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Test Accuracy of the model on the 10000 test images: {} %'
    .format((correct / total) * 100))

Test Accuracy of the model on the 10000 test images: 90.0 %


In [14]:
def test_image(image_name, category):
    img = cv2.imread("../concat_data/%s/%s" %(category, image_name), 0)
    # print(img)
    img = torch.tensor(img, dtype=torch.float32)
    labels = {'focused': 0,
          'happy': 1,
          'neutral': 2,
          'surprised': 3}
    images_test_tensor = torch.tensor(img, dtype=torch.float32)
    images_test_tensor = images_test_tensor.unsqueeze(1)
    labels_test_tensor = torch.tensor(labels[category], dtype=torch.long)

    # Create a TensorDataset
    dataset = td.TensorDataset(images_test_tensor, labels_test_tensor)

    # Create DataLoader for batching and shuffling
    batch_size = 32
    data_loader = td.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    for i, (images, labels) in enumerate(data_loader):
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())
    model.eval()
    with torch.no_grad():
        output = model(images_test_tensor)
        _, predicted = torch.max(output.data, 1)
        print(predicted)
        print(category)
        print(image_name)
        print("///////")
        return predicted

In [15]:
test_image("246_chicago-faces.jpg", "happy")

C:\Users\Luis\AppData\Local\Temp\ipykernel_17924\3217533182.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images_test_tensor = torch.tensor(img, dtype=torch.float32)


IndexError: Dimension specified as 0 but tensor has no dimensions